## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
#%matplotlib qt

# Calibration

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
print('{} images loaded'.format(len(images)))

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #print(gray)
    #plt.imshow(img)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(50)
#print(img.shape)
#cv2.destroyAllWindows()

img_shape = gray.shape[::-1]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape, None, None)
print('calibration done')

20 images loaded
calibration done


# Functions

In [3]:
# Ringbuffer Class imported

class RingBuffer:
    """ class that implements a not-yet-full buffer """
    def __init__(self,size_max):
        self.max = size_max
        self.data = []

    class __Full:
        """ class that implements a full buffer """
        def append(self, x):
            """ Append an element overwriting the oldest one. """
            self.data[self.cur] = x
            self.cur = (self.cur+1) % self.max
        def get(self):
            """ return list of elements in correct order """
            return self.data[self.cur:]+self.data[:self.cur]

    def append(self,x):
        """append an element at the end of the buffer"""
        self.data.append(x)
        if len(self.data) == self.max:
            self.cur = 0
            # Permanently change self's class from non-full to full
            self.__class__ = self.__Full

    def get(self):
        """ Return a list of elements from the oldest to the newest. """
        return self.data


In [4]:
# Define a class to receive the characteristics of each line detection
class Line:
    def __init__(self, bufferSize):
        # was the line detected in the last iteration?
        self.detected = False
        # x values of the last n fits of the line
        self.recent_xfitted = None#RingBuffer(self.n)
        #average x values of the fitted line over the last n iterations
        self.bestx = None
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = [np.array([False])]
        #polynomial coefficients for the most recent fit
        self.recent_fits = RingBuffer(bufferSize)#[np.array([False])]
        self.current_fit = [np.array([False])]
        #radius of curvature of the line in some units
        self.list_curv_left = RingBuffer(bufferSize)
        self.list_curv_right = RingBuffer(bufferSize)
        self.radius_of_curvature = None
        #distance in meters of vehicle center from the line
        self.line_base_pos = None
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')
        #x values for detected line pixels
        self.allx = None
        #y values for detected line pixels
        self.ally = None
        self.offset = None
    
    def update_filter(self):
        self.recent_fits.append(self.current_fit)
        self.best_fit = np.mean(self.recent_fits.data, axis=0)
        
    def update_curv(self, curv_l, curv_r):
        print(curv_l)
        print(curv_r)
        self.list_curv_left.append(curv_l)
        self.list_curv_right.append(curv_r)
        self.radius_of_curvature = np.mean(np.vstack((self.list_curv_left.data, self.list_curv_right.data)))

In [25]:
def plot_comparison(img1, img2, title='comparison'):
    '''
    plots two images next to each other with given title
    '''
    if np.max(img1)<=1:
        img1*=255
    if np.max(img2)<=1:
        img2*=255
    comparison = np.hstack((img1, img2))
    cv2.imshow(title, comparison)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def undistort(img, mtx, dist, visu=False):
    '''
    undistorts image, visualization optional
    '''
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    if visu:
        plot_comparison(img, dst, 'undistortion')
        
    return dst

def get_perspective_transform(img_shape, pad_top=0, pad_bottom=10, pad_l=300, pad_r=300):
    '''
    returns perspective transform matrix for warp with padding
    '''
    # source points adjacent to lane lines
    src = np.float32([[700, 450],
                [1130, 660],
                [585, 450],
                [200, 660]])
    # target points on warped image
    dst = np.float32([[img_shape[0]-pad_r, pad_top],
                      [img_shape[0]-pad_r, img_shape[1]-pad_bottom],
                      [pad_l, pad_top],
                      [pad_l, img_shape[1]-pad_bottom]])
    # calc perspective transform
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    return M, M_inv

def birdview_transform(img, M, visu=False):
    '''
    transforms an image to birdview, visualization optional
    '''
    shape = img.shape[1::-1]
    warped = cv2.warpPerspective(img, M, shape, flags=cv2.INTER_LINEAR)
    
    if visu:
        plot_comparison(img, warped, 'warping')
    
    return warped

def hls_select(img, channel='S', thresh=(0, 255), visu=False):
    '''
    returns binary mask by color selection, visualization optional
    '''
    
    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    if channel=='H':
        chn = hls[:,:,0]
    elif channel=='L':
        chn = hls[:,:,1]
    elif channel=='S':
        chn = hls[:,:,2]
    else:
        print('wrong value for channel, select {H,L,S}')
    # Apply a threshold a channel
    # Return a binary image of threshold result
    binary_output = np.zeros_like(chn)
    binary_output[(chn > thresh[0]) & (chn <= thresh[1])] = 1
    
    if visu:
        plot_comparison(chn, binary_output*255, 'HLS color selection')
        
    return binary_output

def dir_sobel_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2), visu=False):
    '''
    returns binary mask by direction of sobel gradient, visualization optional
    '''
    # Apply the following steps to img
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    direction = np.arctan2(abs_sobely, abs_sobelx)
    # Create a binary mask where direction thresholds are met
    # Return this mask as your binary_output image
    binary_output = np.zeros_like(direction)
    binary_output[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
    
    if visu:
        plot_comparison(gray, binary_output*255, 'gradient direction selection')
        
    return binary_output

def abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=100, visu=False):
    '''
    returns binary mask by directional sobel gradient, visualization optional
    '''
    # Apply the following steps to img
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take the derivative in x or y given orient = 'x' or 'y'
    if not orient == 'x' or orient =='y':
        print("ERROR: Wrong orientation. Choose 'x' or 'y'!")
    sobel = cv2.Sobel(gray, cv2.CV_64F, orient == 'x', orient == 'y')
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a mask of 1's where the scaled gradient magnitude 
    # is > thresh_min and < thresh_max
    # Return this mask as your binary_output image
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    if visu:
        plot_comparison(gray, binary_output*255, 'gradient x/y selection')
        
    return binary_output

def mag_sobel_thresh(img, sobel_kernel=3, mag_thresh=(0, 255), visu=False):
    '''
    returns binary mask by magnitude of sobel gradient, visualization optional
    '''
    # Apply the following steps to img
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the magnitude
    mag_sobel = np.sqrt(np.square(sobelx) + np.square(sobely))
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*mag_sobel/np.max(mag_sobel))
    # Create a binary mask where mag thresholds are met
    # Return this mask as your binary_output image
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    
    if visu:
        plot_comparison(gray, binary_output*255, 'gradient magnitude selection')
        
    return binary_output

def check_non_binary(img, false=0, true=255):
    '''
    checks for non-binary image values
    '''
    small = np.zeros_like(img)
    small[(img<true) & (img>false)] = 1
    print(np.any(small))
    
def preproc_image(image, mtx, dist, M):
    '''
    preprocesses input image (RGB)
    '''
    undist_road = undistort(image, mtx, dist, visu=False)
    # masking
    dir_binary = dir_sobel_thresh(undist_road, sobel_kernel=15, thresh=(0.7, 1.3), visu=False)
    abs_binary = abs_sobel_thresh(undist_road, orient='x', thresh_min=40, thresh_max=100, visu=False)
    #mag_binary = mag_sobel_thresh(road_img, sobel_kernel=3, mag_thresh=(30, 100), visu=True)
    hls_binary = hls_select(undist_road, 'S', thresh=(100, 255), visu=False)
    #hls_binary2 = hls_select(undist_road, 'H', thresh=(15, 100), visu=True)
    # combine binary masks
    binary_combined = np.zeros_like(hls_binary)
    binary_combined[((abs_binary == 1) | (hls_binary == 1)) & (dir_binary == 1)] = 1
    # 3 channel conversion
    #binary_combined*=255
    #check_non_binary(binary_combined) #####
    #combined3ch = np.dstack((binary_combined, binary_combined, binary_combined))
    #plot_comparison(undist_road, combined3ch, 'combined masks')
    # birdview transform
    binary_warped = birdview_transform(binary_combined, M, visu=False)
    #check_non_binary(binary_warped) #####
    return binary_warped, undist_road

def find_lane_polinomials(binary_warped, nWindows=9):
    '''
    finds polynomials of lane marks using nWindows sliding windows
    '''
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    if leftLine.detected and rightLine.detected:
        centerLane.detected = True
        # search in vicinity of previously detected lines
        margin = 100
        left_lane_inds = ((nonzerox > (leftLine.best_fit[0]*(nonzeroy**2) + leftLine.best_fit[1]*nonzeroy + 
            leftLine.best_fit[2] - margin)) & (nonzerox < (leftLine.best_fit[0]*(nonzeroy**2) + 
            leftLine.best_fit[1]*nonzeroy + leftLine.best_fit[2] + margin))) 

        right_lane_inds = ((nonzerox > (rightLine.best_fit[0]*(nonzeroy**2) + rightLine.best_fit[1]*nonzeroy + 
            rightLine.best_fit[2] - margin)) & (nonzerox < (rightLine.best_fit[0]*(nonzeroy**2) + 
            rightLine.best_fit[1]*nonzeroy + rightLine.best_fit[2] + margin)))
    else:
        # search from ground up
        print('initializing')
        centerLane.detected = False
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
        #plt.figure(figsize=(15,8))
        #plt.imshow(binary_warped*255, cmap='gray')
        #bar = plt.bar(range(len(histogram)), histogram)
        #plt.show()
    
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nWindows)
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
    
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
    
        # Create empty lists to receive left and right lane pixel indices  
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        cntWinLeft = 0
        cntWinRight = 0
        cntWinTresh = 6
        for window in range(nWindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
            (100,255,100), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
            (100,255,100), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
                cntWinLeft += 1
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
                cntWinRight += 1

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        # Check for detection
        if cntWinLeft >= cntWinTresh:
            leftLine.detected = True
            print('left line initialized')
        if cntWinRight >= cntWinTresh:
            rightLine.detected = True
            print('right line initialized')
        
    # Extract left and right line pixel positions
    leftLine.allx = nonzerox[left_lane_inds]
    leftLine.ally = nonzeroy[left_lane_inds] 
    rightLine.allx = nonzerox[right_lane_inds]
    rightLine.ally = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    leftLine.current_fit = np.polyfit(leftLine.ally, leftLine.allx, 2)
    rightLine.current_fit = np.polyfit(rightLine.ally, rightLine.allx, 2)
    return out_img

def get_result(road_image, out_image, M_inv):
    # Generate x and y values for plotting
    ploty = np.linspace(0, out_image.shape[0]-1, out_image.shape[0] )
    leftLine.bestx = leftLine.best_fit[0]*ploty**2 + leftLine.best_fit[1]*ploty + leftLine.best_fit[2]
    leftLine.bestx[leftLine.bestx >= road_image.shape[1]] = road_image.shape[1]-1
    rightLine.bestx = rightLine.best_fit[0]*ploty**2 + rightLine.best_fit[1]*ploty + rightLine.best_fit[2]
    rightLine.bestx[rightLine.bestx >= road_image.shape[1]] = road_image.shape[1]-1

    # highlight detections
    # Identify the x and y positions of all nonzero pixels in the image
    #nonzero = binary_warped.nonzero()
    #nonzeroy = np.array(nonzero[0])
    #nonzerox = np.array(nonzero[1])
    out_image[leftLine.ally, leftLine.allx] = [255, 50, 50]
    out_image[rightLine.ally, rightLine.allx] = [50, 50, 255]
    #print(left_fitx)
    #print(out_img.shape)
    out_image[ploty.astype(int), leftLine.bestx.astype(int)] = [255, 255, 0]
    out_image[ploty.astype(int), rightLine.bestx.astype(int)] = [255, 255, 0]

    if centerLane.detected:
        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        margin = 100
        left_line_window1 = np.array([np.transpose(np.vstack([leftLine.bestx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([leftLine.bestx+margin, 
                                      ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([rightLine.bestx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([rightLine.bestx+margin, 
                                      ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Create an image to show the selection window
        window_img = np.zeros_like(out_image)
        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_image, 1, window_img, 0.3, 0)
        #plt.figure(num=1, figsize=(15, 15))
        #plt.imshow(result_road)
        ##plt.plot(left_fitx, ploty, color='yellow')
        ##plt.plot(right_fitx, ploty, color='yellow')
        #plt.show()
    else:
        result = out_image
        #plt.figure(num=1, figsize=(15, 15))
        #plt.imshow(result_road)
        ##plt.plot(left_fitx, ploty, color='yellow')
        ##plt.plot(right_fitx, ploty, color='yellow')
        #plt.show()
    result_road = birdview_transform(result, M_inv)
    lanes = cv2.addWeighted(road_image, 1, result_road, 1, 0)
    return lanes#, result

def draw_lane(road_image, M_inv):
    # Create an image to draw the lines on
    #warp_zero = np.zeros_like(road_image).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    color_warp = np.zeros_like(road_image)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, road_image.shape[0]-1, road_image.shape[0] )
    leftLine.bestx = leftLine.best_fit[0]*ploty**2 + leftLine.best_fit[1]*ploty + leftLine.best_fit[2]
    leftLine.bestx[leftLine.bestx >= road_image.shape[1]] = road_image.shape[1]-1
    rightLine.bestx = rightLine.best_fit[0]*ploty**2 + rightLine.best_fit[1]*ploty + rightLine.best_fit[2]
    rightLine.bestx[rightLine.bestx >= road_image.shape[1]] = road_image.shape[1]-1
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([leftLine.bestx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rightLine.bestx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, (road_image.shape[1], road_image.shape[0])) 
    # Combine the result with the original image
    lanes = cv2.addWeighted(road_image, 1, newwarp, 0.3, 0)
    return lanes

def get_curv(image):
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0] )
    y_eval = np.max(ploty)
    #print(len(ploty))
    #pprint(len(leftLine.bestx))
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftLine.bestx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightLine.bestx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    leftLine.radius_of_curvature = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    rightLine.radius_of_curvature = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    centerLane.update_curv(leftLine.radius_of_curvature, rightLine.radius_of_curvature)
    #print(leftLine.radius_of_curvature, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m

def calc_vehicle_offset(image):
    left_fit = leftLine.best_fit
    right_fit = rightLine.best_fit
    y = image.shape[0] - 1
    x_left = left_fit[0]*(y**2) + left_fit[1]*y + left_fit[2]
    x_right = right_fit[0]*(y**2) + right_fit[1]*y + right_fit[2]
    vehicle_offset = image.shape[1]/2 - (x_left + x_right)/2
    # Convert pixel offset to meters
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    vehicle_offset *= xm_per_pix
    centerLane.offset = vehicle_offset
    
def addText(image):
    distanceFromCenterline = centerLane.offset
    curvature = centerLane.radius_of_curvature
    if(curvature <= 1500):
        line1 = "Curvature Radius: %5d [m]" % (curvature)
    else:
        line1 = "Curvature Radius: None"
    line2 = "Lane Offset: %8.2f [m]" % (distanceFromCenterline)
    text_image =cv2.putText(img=np.copy(image), text=line1, org=(50,50), fontFace=1, fontScale=2, color=(255,255,255), thickness=2)
    text_image =cv2.putText(text_image, text=line2, org=(50,100), fontFace=1, fontScale=2, color=(255,255,255), thickness=2)
    return text_image
    
def process_image(image, M, M_inv):
    # preprocessing
    binary_warped, undistorted = preproc_image(image, mtx, dist, M)
    # lane finding
    out_image = find_lane_polinomials(binary_warped, nWindows=9)
    leftLine.update_filter()
    rightLine.update_filter()
    #lanes = get_result(undistorted, out_image, M_inv)
    lanes = draw_lane(undistorted, M_inv)
    get_curv(out_image)
    calc_vehicle_offset(out_image)
    lanes = addText(lanes)
    return lanes

# Tests

# Pipeline
## Init

In [26]:
leftLine = Line(5)
rightLine = Line(5)
centerLane = Line(5)

## Run

In [27]:
# read image
road_img = cv2.imread('test_images/test2.jpg')
shape_img = road_img.shape[1::-1]
M, M_inv = get_perspective_transform(shape_img)
# preprocessing
lane = process_image(road_img, M, M_inv)
cv2.imshow('lane', lane)
cv2.waitKey(0)
cv2.destroyAllWindows()

initializing
left line initialized
right line initialized
516.350518152
401.869736354


## Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
leftLine = Line(5)
rightLine = Line(5)
centerLane = Line(5)
visualize = True

In [ ]:
cap = cv2.VideoCapture("input_videos\\project_video.mp4")
shape_img = cap.read()[1].shape[1::-1]
M, M_inv = get_perspective_transform(shape_img)
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter("output_videos\\project_video.mp4",fourcc, 20.0, shape_img)
while(cap.isOpened()):
    ff,frame = cap.read()
    if(ff == True):
        lanes = process_image(frame, M, M_inv)
        out.write(lanes)
        if(visualize == True):
            cv2.imshow('frame',lanes)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("input_videos/project_video.mp4"))

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos/project_video.mp4"))

In [ ]:
img = cv2.imread('test_images/test6.jpg')
img1 = undistort(img, mtx, dist, True)

In [ ]:
img = cv2.imread('test_images/test2.jpg')
img1 = preproc_image(img, mtx, dist, M)